# [문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough
- itemgetter()

세션ID 설정하고, 대화를 진행합니다. (multi-turn conversation)
LLM 모델이 과거 대화를 알지(기억) 못하는 상황을 만드세요.


In [11]:
## 1. 모듈 import
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate

## [방법 1]
# from langchain_core.chat_history import BaseChatMessageHistory
# from langchain_core.chat_history import InMemoryChatMessageHistory

## [방법 2]
# from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory

## [방법 3]
from langchain_core.chat_history import (
    BaseChatMessageHistory, 
    InMemoryChatMessageHistory
)


## 2. .env 파일에서 환경변수 읽어오기
load_dotenv()


True

In [12]:

## 3. 세션별 대화 히스토리를 저장할 임시 메모리 저장소
## type: dict 
store = {}

## 4. 함수 정의: 세션 ID에 따라 대화 히스토리 반환
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

## 5. prompt template 정의
messages = [
    ('system', '''당신은 이력서 작성 컨설턴트입니다.
아래 정보를 바탕으로 지원자 입장에서 2000자 이내로 이력서를 작성합니다.
문장은 자연스럽고 매끄럽게 작성합니다.'''),
	('placeholder', '{chat_history}'),
	('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(messages=messages)


In [13]:

## 6. ChatOpenAI 인스턴스 생성: 모델 생성
llm = ChatOpenAI(
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)


In [14]:
# 7. trim 설정
from langchain_core.messages import trim_messages

trimmer = trim_messages(
    max_tokens= 65,
    strategy='last',
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on='human',
)

In [15]:

## 8. chain 구성
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

chain = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history')| trimmer)
    |prompt 
    | llm 
)

chain

RunnableAssign(mapper={
  chat_history: RunnableLambda(itemgetter('chat_history'))
                | RunnableLambda(...)
})
| ChatPromptTemplate(input_variables=['query'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, T

In [20]:
## 8. 함수 정의: 대화 이력(히스토리) 인스턴스 리턴

def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [21]:

## 8. chain에 대화 히스토리 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key='query',
    history_messages_key='chat_history'
)

## 9. chain 실행
while True:
    query = input('이력서 작성 컨설턴트입니다. 질문하세요. [종료: S] >>> ')

    if query.upper() == 'S':
        break

    with_message_history.invoke(
        {'query': query},
        config={'configurable': {'session_id': 'resume'}}      
    )

[이력서]

**개인 정보**
- 이름: 유명성
- 이메일: ymseong@email.com
- 전화번호: 010-1234-5678
- 주소: 서울특별시 강남구

---

**학력**

- **서울대학교**, 서울특별시  
  경영학 학사  
  2016년 3월 – 2020년 2월  

- **서울과학기술고등학교**, 서울특별시  
  졸업  
  2013년 3월 – 2016년 2월  

---

**경력**

- **ABC 마케팅**, 서울특별시  
  마케팅 코디네이터  
  2021년 1월 – 현재  
  - 디지털 마케팅 캠페인 기획 및 실행
  - SNS 플랫폼을 활용한 브랜드 인지도 제고
  - 고객 설문조사를 통한 시장 트렌드 분석 및 전략 수립

- **XYZ 컨설팅**, 서울특별시  
  인턴  
  2019년 6월 – 2019년 8월  
  - 프로젝트 관리 및 클라이언트 미팅 지원
  - 시장 조사 및 데이터 분석 보조
  - 보고서 작성 및 프레젠테이션 준비 지원

---

**기술**

- Microsoft Office Suite (Word, Excel, PowerPoint)
- Adobe Photoshop 및 Illustrator
- Google Analytics
- SEO 및 SEM 전략

---

**언어**

- 한국어: 모국어
- 영어: 고급 (TOEIC 920점)

---

**수상 및 인증**

- 서울대학교 총장상 (2019년 우수 학업 성취 및 리더십)
- Google Analytics 인증 (2021년)
- 국제마케팅협회 주최 마케팅 아이디어 경진대회 우승 (2020년)

---

**기타 활동**

- **서울대학교 광고동아리**  
  회장  
  2018년 3월 – 2019년 2월
  - 동아리 활동 총괄 및 외부 행사 기획
  - 기업과의 협력 프로젝트 진행

- **봉사활동**, 서울시 청소년 센터  
  멘토  
  2020년 3월 – 2020년 12월  
  - 고등학생 대상 진로 상담 및 학습 지도
  -

In [22]:
get_history('resume')

InMemoryChatMessageHistory(messages=[HumanMessage(content='이름:유명성', additional_kwargs={}, response_metadata={}), AIMessage(content='[이력서]\n\n**개인 정보**\n- 이름: 유명성\n- 이메일: ymseong@email.com\n- 전화번호: 010-1234-5678\n- 주소: 서울특별시 강남구\n\n---\n\n**학력**\n\n- **서울대학교**, 서울특별시  \n  경영학 학사  \n  2016년 3월 – 2020년 2월  \n\n- **서울과학기술고등학교**, 서울특별시  \n  졸업  \n  2013년 3월 – 2016년 2월  \n\n---\n\n**경력**\n\n- **ABC 마케팅**, 서울특별시  \n  마케팅 코디네이터  \n  2021년 1월 – 현재  \n  - 디지털 마케팅 캠페인 기획 및 실행\n  - SNS 플랫폼을 활용한 브랜드 인지도 제고\n  - 고객 설문조사를 통한 시장 트렌드 분석 및 전략 수립\n\n- **XYZ 컨설팅**, 서울특별시  \n  인턴  \n  2019년 6월 – 2019년 8월  \n  - 프로젝트 관리 및 클라이언트 미팅 지원\n  - 시장 조사 및 데이터 분석 보조\n  - 보고서 작성 및 프레젠테이션 준비 지원\n\n---\n\n**기술**\n\n- Microsoft Office Suite (Word, Excel, PowerPoint)\n- Adobe Photoshop 및 Illustrator\n- Google Analytics\n- SEO 및 SEM 전략\n\n---\n\n**언어**\n\n- 한국어: 모국어\n- 영어: 고급 (TOEIC 920점)\n\n---\n\n**수상 및 인증**\n\n- 서울대학교 총장상 (2019년 우수 학업 성취 및 리더십)\n- Google Analytics 인증 (2021년)\n- 국제마케팅협회 주최 마케팅 아이디어 경진대회 우승 (2020년)\n\n---

In [23]:
for info in get_history('resume').messages:
    print(info.content)
    print('*' * 50)

이름:유명성
**************************************************
[이력서]

**개인 정보**
- 이름: 유명성
- 이메일: ymseong@email.com
- 전화번호: 010-1234-5678
- 주소: 서울특별시 강남구

---

**학력**

- **서울대학교**, 서울특별시  
  경영학 학사  
  2016년 3월 – 2020년 2월  

- **서울과학기술고등학교**, 서울특별시  
  졸업  
  2013년 3월 – 2016년 2월  

---

**경력**

- **ABC 마케팅**, 서울특별시  
  마케팅 코디네이터  
  2021년 1월 – 현재  
  - 디지털 마케팅 캠페인 기획 및 실행
  - SNS 플랫폼을 활용한 브랜드 인지도 제고
  - 고객 설문조사를 통한 시장 트렌드 분석 및 전략 수립

- **XYZ 컨설팅**, 서울특별시  
  인턴  
  2019년 6월 – 2019년 8월  
  - 프로젝트 관리 및 클라이언트 미팅 지원
  - 시장 조사 및 데이터 분석 보조
  - 보고서 작성 및 프레젠테이션 준비 지원

---

**기술**

- Microsoft Office Suite (Word, Excel, PowerPoint)
- Adobe Photoshop 및 Illustrator
- Google Analytics
- SEO 및 SEM 전략

---

**언어**

- 한국어: 모국어
- 영어: 고급 (TOEIC 920점)

---

**수상 및 인증**

- 서울대학교 총장상 (2019년 우수 학업 성취 및 리더십)
- Google Analytics 인증 (2021년)
- 국제마케팅협회 주최 마케팅 아이디어 경진대회 우승 (2020년)

---

**기타 활동**

- **서울대학교 광고동아리**  
  회장  
  2018년 3월 – 2019년 2월
  - 동아리 활동 총괄 및 외부 행사 기획
  - 기업과의 협력 프로젝트 진행

- **봉사활동**, 서울시 청소년 센터  
  멘

In [24]:
for message in get_history('resume').messages:
    print(f'[{message.type.upper()}]: {message.content}\n')

[HUMAN]: 이름:유명성

[AI]: [이력서]

**개인 정보**
- 이름: 유명성
- 이메일: ymseong@email.com
- 전화번호: 010-1234-5678
- 주소: 서울특별시 강남구

---

**학력**

- **서울대학교**, 서울특별시  
  경영학 학사  
  2016년 3월 – 2020년 2월  

- **서울과학기술고등학교**, 서울특별시  
  졸업  
  2013년 3월 – 2016년 2월  

---

**경력**

- **ABC 마케팅**, 서울특별시  
  마케팅 코디네이터  
  2021년 1월 – 현재  
  - 디지털 마케팅 캠페인 기획 및 실행
  - SNS 플랫폼을 활용한 브랜드 인지도 제고
  - 고객 설문조사를 통한 시장 트렌드 분석 및 전략 수립

- **XYZ 컨설팅**, 서울특별시  
  인턴  
  2019년 6월 – 2019년 8월  
  - 프로젝트 관리 및 클라이언트 미팅 지원
  - 시장 조사 및 데이터 분석 보조
  - 보고서 작성 및 프레젠테이션 준비 지원

---

**기술**

- Microsoft Office Suite (Word, Excel, PowerPoint)
- Adobe Photoshop 및 Illustrator
- Google Analytics
- SEO 및 SEM 전략

---

**언어**

- 한국어: 모국어
- 영어: 고급 (TOEIC 920점)

---

**수상 및 인증**

- 서울대학교 총장상 (2019년 우수 학업 성취 및 리더십)
- Google Analytics 인증 (2021년)
- 국제마케팅협회 주최 마케팅 아이디어 경진대회 우승 (2020년)

---

**기타 활동**

- **서울대학교 광고동아리**  
  회장  
  2018년 3월 – 2019년 2월
  - 동아리 활동 총괄 및 외부 행사 기획
  - 기업과의 협력 프로젝트 진행

- **봉사활동**, 서울시 청소년 센터  
  멘토  
  2020년 3월 – 2020년 12월  
  - 고등